### **Jina Reranker**

API 키당 100만 토큰 제공 

- [API 키 발급](https://jina.ai/reranker)

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# 문서 로드
documents = TextLoader("./data/appendix-keywords.txt").load()

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
texts = text_splitter.split_documents(documents)

# 검색기 초기화
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever(
    search_kwargs={"k": 10}
)

# 질의문
query = "Word2Vec 에 대해서 설명해줘."

# 문서 검색
docs = retriever.invoke(query)

# 문서 출력
pretty_print_docs(docs)

In [ ]:
from ast import mod
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import JinaRerank

# JinaRerank 압축기 초기화
compressor = JinaRerank(model="jina-reranker-v2-base-multilingual", top_n=3)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 관련 문서 검색 및 압축
compressed_docs = compression_retriever.invoke("Word2Vec 에 대해서 설명해줘.")